In [1]:
import ee
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import geemap.foliumap as geemap

ee.Authenticate()
ee.Initialize()

In [25]:
s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD').filterMetadata('instrumentMode', 'equals', 'IW').filter(ee.Filter.eq('transmitterReceiverPolarisation', ['VV', 'VH'])).sort('system:time')
s2_collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
states = ee.FeatureCollection('TIGER/2018/States')

start_date = '2018-01-01'
end_date = '2018-12-31'

aoi = 'WA'
step = 10

state = states.filter(ee.Filter.eq('STUSPS', aoi)).geometry()


In [26]:
def toNatural(img: ee.Image) -> ee.Image:
  return ee.Image(10.0).pow(img.select('..').divide(10.0)).copyProperties(img, ['system:time_start'])

def toDB(img: ee.Image) -> ee.Image:
  return ee.Image(img).log10().multiply(10.0).copyProperties(img, ['system:time_start'])

def maskEdge(img: ee.Image) -> ee.Image:
  mask = img.select(0).unitScale(-25, 5).multiply(255).toByte().connectedComponents(ee.Kernel.rectangle(1,1), 100)
  return img.updateMask(mask.select(0).abs());  

def step_mean(range, collection) -> ee.Image:
  dstamp = ee.Date(ee.List(range).get(0)).format('YYYYMMdd')
  temp_collection = collection.filterDate(ee.List(range).get(0),
  ee.List(range).get(1)).mean().select(['VH', 'VV'], [ee.String('VV_').cat(dstamp), ee.String('VH_').cat(dstamp)])
  return temp_collection


In [38]:
# Olha Danylo's procedure to create weekly means (adapted)
days = ee.List.sequence(0, ee.Date(end_date).difference(ee.Date(start_date), 'day'), step).map(lambda x: ee.Date(start_date).advance(x, "day"))
dates = days.slice(0,-1).zip(days.slice(1))
s1res = dates.map(lambda x: step_mean(x, s1_collection))
s1res = s1res.map(lambda x: toDB(x))

def stack(i1, i2):
  return ee.Image(i1).addBands(ee.Image(i2))

s1stack = s1res.slice(1).iterate(stack, s1res.get(0))
s1stack = ee.Image(s1stack)
s1stack = s1stack.toFloat()
s1stack = s1stack.clip(state)

viz ={'bands':['VV_20180511','VV_20180610','VV_20180710'], 'min':-25, 'max':0}

Map = geemap.Map(center=[40,-100], zoom=4)
Map.addLayer(s1stack,viz,'s1stack_europe',0)
Map

In [33]:
info = s1stack.getInfo()

In [37]:
type(info['bands'])

list